In [ ]:
import tkinter as tk
import mysql.connector
import re
from tkinter import ttk
from tkinter import messagebox as msb
# from openpyxl import load_workbook

# Khởi tạo các hàm của chương trình

# def connect_db():
#     mydb = mysql.connector.connect(host='localhost', database='mysql',user='root', password='cqphuc@3009')
#     return mydb    

def check_blank_data():
    # Kiểm tra các trường nhập có bị trống hay không
    errors = []
    message=''
    if entry_value_id.get()=='':
        errors.append('ID')
    if entry_value_age.get()=='':
        errors.append('Age')
    if entry_value_fname.get()=='':
        errors.append('First name')
    if entry_value_lname.get()=='':
        errors.append('Last name')
    if entry_value_phone.get()=='':
        errors.append('Phone')
    if cbx_selected_class.get()=='--- Select a class ---':
        errors.append('Class')
    for error_index in range(0,len(errors)):
        if error_index == len(errors)-1:
            message += errors[error_index] + ' '
        else:    
            message += errors[error_index] + ', '
    if message =='':
        return True
    else:
        msb.showwarning('Warnning',message+'is blank!')
        return False

def validate_phone(value):
    # Bắt lỗi nhập số điện thoại
    # '^0': ký tự đầu phải là 0; 'd{9}': yêu cầu phải là số 
    pattern = '^0\d{9}'
    if entry_value_phone.get() !='':
        if re.fullmatch(pattern, entry_value_phone.get()) is None:
            return False
        else:
            return True
    
def validate_age(value):
    # Bắt lỗi nhập tuổi
    pattern = '\d+'
    if entry_value_age.get() !='':
        if re.fullmatch(pattern, entry_value_age.get()) is None:
            return False
        else:
            return True

def on_invalid_phone():
    # Hiển thị thông báo khi người dùng nhập điện thoại sai định dạng 
    msb.showwarning('Warnning','Invalid phone number!\nIt should be like 0xxxxxxxxx') 
        
def on_invalid_age():
    # Hiển thị thông báo khi người dùng nhập tuổi sai định dạng 
    msb.showwarning('Warnning','Invalid Age!\nIt should be number which is greater than zero')   

def clear_data():
    # Làm trống dữ liệu trên các trường thông tin
    entry_value_id.set('')
    entry_value_age.set('')
    entry_value_fname.set('')
    entry_value_lname.set('')
    entry_value_phone.set('')
    cbx_class.current(0)

def item_selected():
    # Lấy dữ liệu của học viên được chọn từ treeview
    selected_item = tvw_info.selection()
    item = tvw_info.item(selected_item,option='values')
    return item

def view_student():
    # Hiển thị thông tin của sinh viên được chọn lên các trường thông tin
    try:
        item = item_selected()
        entry_value_id.set(item[0])
        entry_value_fname.set(item[1])
        entry_value_lname.set(item[2])
        entry_value_age.set(item[3])
        entry_value_phone.set(item[4])
        cbx_class.current(cbx_class['value'].index(item[5]))
    except:
        clear_data()

def view_all_student(): 
    # Hiển thị danh sách sinh viên lên treeview
#     mydb = connect_db()
#     mycursor = mydb.cursor()
    mycursor.execute("SELECT * FROM msd_student")
    student_list = mycursor.fetchall()
    mycursor.execute("SELECT count(*) FROM information_schema.columns WHERE table_name = 'msd_student'")
    colnum = mycursor.fetchone()[0]
    
    for item in tvw_info.get_children():
        tvw_info.delete(item)
    for student in student_list:
        tvw_value = []
        for item in range(0,colnum):
            tvw_value.append(student[item])
        tvw_info.insert('',tk.END,value=tvw_value)

def add_student():
    # Thêm mới học viên
    new_id=entry_value_id.get()    
    new_age = ''
    if validate_age == False:
        entry_value_age.set('')
    new_age=entry_value_age.get()    
    new_fname=entry_value_fname.get()    
    new_lname=entry_value_lname.get()    
    new_phone = ''
    if validate_phone == False:        
        entry_value_phone.set('')
    new_phone=entry_value_phone.get()    
    new_class=cbx_selected_class.get()    
#     new_student=[new_id,new_fname,new_lname,new_age,new_phone,new_class]
    insert_query = "INSERT INTO msd_student VALUES (%s, %s, %s, %s, %s, %s)"
    insert_value = new_id,new_lname,new_fname,new_age,new_phone,new_class
    
#     mydb = connect_db()
#     mycursor = mydb.cursor()

    if check_blank_data() == True:        
        # Kiểm tra xem ID có bị trùng không
        mycursor.execute("SELECT stu_id FROM msd_student")
        students_id = mycursor.fetchall()
        for stu_id in students_id:
            if stu_id[0] == new_id:
                msb.showwarning('Warnning','ID is existed!')
                return 
#         Thực hiện thêm vào file excel nếu không trùng
        mycursor.execute(insert_query,insert_value)
        mydb.commit()
        msb.showinfo('Anoucement','Add successful!')
        clear_data()
        view_all_student()
    
def update_student():
    # Sửa thông tin học viên
    update_id = entry_value_id.get()
    update_age = ''
    if validate_age == False:
        entry_value_age.set('')
    update_age=entry_value_age.get()    
    update_fname=entry_value_fname.get()    
    update_lname=entry_value_lname.get()    
    update_phone = ''
    if validate_phone == False:        
        entry_value_phone.set('')
    update_phone=entry_value_phone.get()    
    update_class=cbx_selected_class.get()    
#     update_student=[update_id,update_fname,update_lname,update_age,update_phone,update_class]
    
    update_query = f"UPDATE msd_student SET l_name={update_lname}, f_name={update_fname}, age={update_age}, phone={update_phone}, class={update_class} WHERE stu_id={update_id}"
    mycursor.execute(update_query)  
    mydb.commit()
    
    msb.showinfo('Anoucement','Update successful!')
    clear_data()        
    view_all_student()
    
def delete_student():
#     view_student()
    item = item_selected()
    delete_id = item[0]
    answer = msb.askyesno('Confirm','Do you want to delete?')
    if answer == 1:
        delete_query = "DELETE FROM msd_student WHERE stu_id = %s"
        mycursor.execute(delete_query,delete_id)
        mydb.commit()
        msb.showinfo('Anoucement','Delete successful!')        
    view_all_student()
        
def delete_all_student():
    answer = msb.askyesno('Confirm','Do you want to delete?')
    if answer == 1:
        max_row = ws.max_row+1
        for row in range(2,max_row):
            ws.delete_rows(2)
        msb.showinfo('Anoucement','Delete all successful!')  
    
    view_all_student()

def search():
    tvw_value = []
    index = -1
    for item in tvw_info.get_children():
                tvw_info.delete(item)
    if cbx_selected_search.get() == 'ID':
        index = 0
    if cbx_selected_search.get() == 'First Name':
        index = 1
    if cbx_selected_search.get() == 'Last Name':
        index = 2
    if cbx_selected_search.get() == 'Age':
        index = 3
    if cbx_selected_search.get() == 'Phone':
        index = 4
    if cbx_selected_search.get() == 'Class':
        index = 5
    for row in ws:
        if row[index].value == entry_value_search.get():
            for col in range(1,ws.max_column+1):
                tvw_value.append(ws.cell(row[index].row,column=col).value)
            tvw_info.insert('',tk.END,value=tvw_value)

def sort():
    sort_list = []  
    index = -1
    sort_type = False
    for item in tvw_info.get_children():
        tvw_info.delete(item)
    if cbx_selected_sort.get() == 'ID':
        index = 0
    if cbx_selected_sort.get() == 'First Name':
        index = 1
    if cbx_selected_sort.get() == 'Last Name':
        index = 2
    if cbx_selected_sort.get() == 'Age':
        index = 3
    if cbx_selected_sort.get() == 'Phone':
        index = 4
    if cbx_selected_sort.get() == 'Class':
        index = 5        
    if cbx_selected_asc.get() == 'Descending':
        sort_type = True

    for row in ws:
        sort_list.append(row[index].value)
    sort_list.pop(0)
    sort_list.sort(reverse=sort_type)
    for item in sort_list:
        for r in range(2,ws.max_row+1):
            tvw_value = []     
            cell_value = ws.cell(row=r,column=index+1).value
            if cell_value == item:
                for c in range(1,ws.max_column+1):
                    tvw_value.append(ws.cell(row=r,column=c).value)
                tvw_info.insert('',tk.END,value=tvw_value)

def exit():
    window.destroy()

# Khởi tạo window chính
window = tk.Tk()
window.title('MSD STUDENT MANAGEMENT')
window.geometry('450x650')
window.resizable(False,False)

# Kết nối mysql
mydb = mysql.connector.connect(host='localhost', database='mysql',user='root', password='cqphuc@3009')
mycursor = mydb.cursor()

# Khởi tạo biến variable của các widget
entry_value_id = tk.StringVar()
entry_value_fname = tk.StringVar()
entry_value_lname = tk.StringVar()
entry_value_age = tk.StringVar()
entry_value_phone = tk.StringVar()
entry_value_search = tk.StringVar()
cbx_selected_search = tk.StringVar()
cbx_selected_class = tk.StringVar()
cbx_selected_sort = tk.StringVar()
cbx_selected_asc = tk.StringVar()
info_column = ('id','l_name','f_name','age','phone','class')


# ===========================================
# Khởi tạo các trường để nhập thông tin
frame_input_info = ttk.Labelframe(window,text='Input information')

label_id = ttk.Label(frame_input_info,text='ID')
label_id.grid(row=0,column=0,sticky='e',padx='5',pady='5')

label_age = ttk.Label(frame_input_info,text='Age')
label_age.grid(row=0,column=2,sticky='e',padx='5',pady='5')

label_lname = ttk.Label(frame_input_info,text='Last Name')
label_lname.grid(row=1,column=0,sticky='e',padx='5',pady='5')

label_fname = ttk.Label(frame_input_info,text='First Name')
label_fname.grid(row=1,column=2,sticky='e',padx='5',pady='5')

label_phone = ttk.Label(frame_input_info,text='Phone.No')
label_phone.grid(row=2,column=0,sticky='e',padx='5',pady='5')

label_class = ttk.Label(frame_input_info,text='Class')
label_class.grid(row=2,column=2,sticky='e',padx='5',pady='5')

entry_id = ttk.Entry(frame_input_info,textvariable=entry_value_id)
entry_id.grid(row=0,column=1,sticky='w',pady='5')

entry_age = ttk.Entry(frame_input_info,textvariable=entry_value_age)
vcmd_age = (entry_age.register(validate_age),'%P')
ivcmd_age = (entry_age.register(on_invalid_age))
entry_age.config(validate='focusout', validatecommand=vcmd_age, invalidcommand=ivcmd_age)
entry_age.grid(row=0,column=3,sticky='w',pady='5')

entry_lname = ttk.Entry(frame_input_info,textvariable=entry_value_lname)
entry_lname.grid(row=1,column=1,sticky='w',pady='5')

entry_fname = ttk.Entry(frame_input_info,textvariable=entry_value_fname)
entry_fname.grid(row=1,column=3,sticky='w',pady='5')

entry_phone = ttk.Entry(frame_input_info,textvariable=entry_value_phone)
vcmd_phone = (entry_phone.register(validate_phone),'%P')
ivcmd_phone = (entry_phone.register(on_invalid_phone))
entry_phone.config(validate='focusout', validatecommand=vcmd_phone, invalidcommand=ivcmd_phone)
entry_phone.grid(row=2,column=1,sticky='w',pady='5')

cbx_class = ttk.Combobox(frame_input_info,state= "readonly",textvariable=cbx_selected_class)
cbx_class['value']=('--- Select a class ---','Mất gốc','Toeic','Luyện đề')
cbx_class.current(0)
cbx_class.grid(row=2,column=3,sticky='w',pady='5')
# cbx_class.bind('<<ComboboxSelected>>', cbx_selected_class)


# ===========================================
# Khởi tạo các Button để thực hiện các chức năng
frame_function = ttk.Labelframe(window,text='Function')

btn_view = ttk.Button(frame_function, text='View All', command=view_all_student)
btn_view.grid(row=0,column=0,sticky='w',padx='5',pady='5')

btn_add = ttk.Button(frame_function, text='Add', command=add_student)
btn_add.grid(row=0,column=1,sticky='w',padx='5',pady='5')

btn_update = ttk.Button(frame_function, text='Update', command=update_student)
btn_update.grid(row=0,column=2,sticky='w',padx='5',pady='5')

btn_delete = ttk.Button(frame_function, text='Delete', command=delete_student)
btn_delete.grid(row=0,column=3,sticky='w',padx='5',pady='5')

btn_delete_all_btn = ttk.Button(frame_function, text='Delete All', command=delete_all_student)
btn_delete_all_btn.grid(row=0,column=4,sticky='w',padx='5',pady='5')

btn_get_info = ttk.Button(frame_function, text='Get Info', command=view_student)
btn_get_info.grid(row=1,column=0,sticky='w',padx='5',pady='5')

btn_clear = ttk.Button(frame_function, text='Clear', command=clear_data)
btn_clear.grid(row=1,column=1,sticky='w',padx='5',pady='5')

btn_exit = ttk.Button(frame_function, text='Exit', command=exit)
btn_exit.grid(row=1,column=2,sticky='w',padx='5',pady='5')


# ===========================================
# Khởi tạo các trường để search
frame_search = ttk.Frame(window)

entry_search = ttk.Entry(frame_search, textvariable=entry_value_search)
entry_search.grid(row=0,column=0,sticky='w',padx='5',pady='5')

cbx_search = ttk.Combobox(frame_search,state= "readonly",textvariable=cbx_selected_search, width=13)
cbx_search['value']=('-- Search by --','ID','Last Name','First Name','Age','Phone','Class')
cbx_search.current(0)
cbx_search.grid(row=0,column=1,sticky='e',pady='5')

btn_search = ttk.Button(frame_search,text='Search',command=search)
btn_search.grid(row=0,column=2,sticky='w',padx='5',pady='5')

# Khởi tạo các trường để sort
cbx_asc = ttk.Combobox(frame_search,state= "readonly",textvariable=cbx_selected_asc, width=11)
cbx_asc['value']=('Ascending','Descending')
cbx_asc.current(0)
cbx_asc.grid(row=1,column=0,sticky='e',padx='5',pady='5')

cbx_sort = ttk.Combobox(frame_search,state= "readonly",textvariable=cbx_selected_sort, width=13)
cbx_sort['value']=('-- Sort by --','ID','Last Name','First Name','Age','Phone','Class')
cbx_sort.current(0)
cbx_sort.grid(row=1,column=1,sticky='e',pady='5')

btn_sort = ttk.Button(frame_search,text='Sort',command=sort)
btn_sort.grid(row=1,column=2,sticky='e',padx='5',pady='5')


# ===========================================
# Khởi tạo các trường để hiển thị thông tin
frame_display_info = ttk.Labelframe(window, text='Display Information')

tvw_info = ttk.Treeview(frame_display_info,column=info_column,show='headings')
tvw_info.column('id',width=30)
tvw_info.column('l_name',width=80)
tvw_info.column('f_name',width=100)
tvw_info.column('age',width=30)
tvw_info.column('phone',width=90)
tvw_info.column('class',width=60)

tvw_info.heading('id',text='ID')
tvw_info.heading('l_name',text='Last Name')
tvw_info.heading('f_name',text='First Name')
tvw_info.heading('age',text='Age')
tvw_info.heading('phone',text='Phone.No')
tvw_info.heading('class',text='Class')

tvw_info.grid(row=1,column=0,sticky='w',padx='5',pady='5')
tvw_info.bind('<<TreeviewSelect>>',item_selected())

scrollbar_y = ttk.Scrollbar(frame_display_info,orient='vertical',command=tvw_info.yview)
tvw_info['yscrollcommand']=scrollbar_y.set
scrollbar_y.grid(row=1,column=1,sticky='w',padx='5',pady='5')

scrollbar_x = ttk.Scrollbar(frame_display_info,orient='horizontal',command=tvw_info.xview)
tvw_info['xscrollcommand']=scrollbar_x.set
scrollbar_x.grid(row=2,column=0,padx='5',pady='5')


# ===========================================
# Sắp xếp các frame
frame_input_info.pack(padx='10',pady='20',anchor='w')
frame_function.pack(padx='10',pady='20',anchor='w')
frame_search.pack(padx='10',pady='0',anchor='w')
frame_display_info.pack(padx='10',pady='0',expand=True,fill='y',anchor='w')

view_all_student()
window.mainloop()